In [1]:


from rcm_library import *

from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

from IPython.display import clear_output

from env import host, username, password


from pandas import value_counts

def get_db_url(database, host=host, user=username, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'




In [2]:
dfa = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])




In [3]:
df=dfa# save point

In [4]:
df['datetime']=df.date +" "+ df.time

df['datetime']=pd.to_datetime(df['datetime'])


In [5]:



df.set_index('datetime',inplace=True)
df.drop(columns=['date','time'],inplace=True)
df.head(3)




,page,id,cohort,ip
datetime,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61


In [6]:
schema='curriculum_logs'
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/{schema}'



query_2 = f'''
SELECT table_name AS "Tables",
ROUND(((data_length + index_length) / 1024 / 1024), 4) AS "Size (MB)"
FROM information_schema.TABLES
WHERE table_schema = "{schema}"
ORDER BY (data_length + index_length) DESC;
'''

info2 = pd.read_sql(query_2, get_db_url(schema))

tablenames = [x[0] for x in [list(i) for i in info2.values]]

x=[(pd.read_sql(f'describe {x}', get_db_url(schema)))for x in tablenames]






display(
    f'In {schema} you have the following table names and their sizes:', info2)
[display(i) for i in x]





'In curriculum_logs you have the following table names and their sizes:'

,Tables,Size (MB)
0,logs,90.6250
1,cohorts,0.0313


,Field,Type,Null,Key,Default,Extra
0,date,text,YES,,None,
1,time,text,YES,,None,
2,path,text,YES,,None,
3,user_id,bigint,YES,,None,
4,cohort_id,double,YES,,None,
5,ip,text,YES,,None,


,Field,Type,Null,Key,Default,Extra
0,id,bigint,YES,MUL,None,
1,name,text,YES,,None,
2,slack,text,YES,,None,
3,start_date,text,YES,,None,
4,end_date,text,YES,,None,
5,created_at,text,YES,,None,
6,updated_at,text,YES,,None,
7,deleted_at,double,YES,,None,
8,program_id,bigint,YES,,None,


[None, None]

In [7]:


# query_logs = '''
# SELECT *
# FROM logs;
# '''
# logs = pd.read_sql(query_logs, url)




# query_cohorts = '''
# SELECT *
# FROM cohorts;
# '''
# cohorts = pd.read_sql(query_cohorts, url)


In [8]:

# sql_query = """
#                 select * from customers 
#                 join contract_types using (contract_type_id)
#                 join internet_service_types using (internet_service_type_id)
#                 join payment_types using (payment_type_id)
#                 """




def curriculum_logs_new():
    '''
    This function reads the curriculum data from the Codeup db into a df.
    '''
    schema='curriculum_logs'
    sql_query=f''' 

    select * from logs as lg
    left join cohorts as cht
    on
    lg.cohort_id = cht.id


    
    
    
    
    
    
    
    '''
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_db_url(schema))
    
    return df




def get_curriculum_logs():
    '''
    This function reads in telco data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    csvname='curriculum_logs.csv'
    if os.path.isfile(csvname):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv(csvname, index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = curriculum_logs_new()
        
        # Cache data
        df.to_csv(csvname)
    
    return df















In [9]:
cur_logs=get_curriculum_logs()

In [10]:
cur_logs.memory_usage().sum() / (1024**2) 

In [11]:
cur_logs.head(5)

cur_logs[(cur_logs.id==cur_logs.cohort_id)]
cur_logs=cur_logs[(cur_logs.id==cur_logs.cohort_id)]




In [12]:
cur_logs

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0


In [13]:

programs=cur_logs[['program_id','cohort_id','start_date',"end_date",'name']]

grp1=programs.groupby(['name','cohort_id','start_date','end_date','program_id'])



In [14]:

metadata=pd.DataFrame(list(grp1.groups.keys()),columns=['name','cohort','start_date','end_date','program_id'])
metadata


,name,cohort,start_date,end_date,program_id
0,Andromeda,31.0,2019-03-18,2019-07-30,2.0
1,Apex,56.0,2020-02-24,2020-07-29,2.0
2,Apollo,9.0,2015-03-30,2015-07-29,4.0
3,Arches,1.0,2014-02-04,2014-04-22,1.0
4,Badlands,2.0,2014-06-04,2014-08-22,1.0
5,Bash,61.0,2020-07-20,2021-01-21,2.0
6,Bayes,34.0,2019-08-19,2020-01-30,3.0
7,Betelgeuse,32.0,2019-05-28,2019-10-08,2.0
8,Ceres,33.0,2019-07-15,2019-12-11,2.0
9,Curie,55.0,2020-02-03,2020-07-07,3.0


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed? 


In [15]:

# grp1keys=tuple(grp1.groups.keys())
# for i in range(len(grp1keys)):
#     # print(grp1.get_group(grp1keys[i]).value_counts(normalize=False).nlargest(1).to_dict())
#     # display(pd.DataFrame(grp1.get_group(grp1keys[i])))
#     display(grp1.get_group(grp1keys[i]).to_dict())





In [16]:

cols=df.columns.to_list();cols



# df.page.


['page', 'id', 'cohort', 'ip']

In [17]:
## the / in page seems to be an error so I am droping those rows from the df for now.

df=df[df.page!='/']


grp1=programs.groupby(['name','cohort_id','start_date','end_date','program_id'])
metadata=pd.DataFrame(list(grp1.groups.keys()),columns=['name','cohort_id','start_date','end_date','program_id'])
typenamecohort=pd.DataFrame(list(metadata.groupby(['cohort_id','name','program_id']).groups.keys()),columns=['cohort_id','name','program_id'])
typenamecohort=typenamecohort.fillna(0)
df=df.fillna(0)


In [18]:
ncol=df.select_dtypes('number').columns.to_list()
df[ncol]=df[ncol].astype(int)


In [19]:


a=set(typenamecohort.cohort_id)
b=set(df.cohort)
c=a.intersection(b)
d=a-c
d

typenamecohort=typenamecohort[typenamecohort.cohort_id!=5]
typenamecohort=typenamecohort.rename(columns={'cohort_id':'cohort'})



In [20]:
typenamecohort


,cohort,name,program_id
0,1.0,Arches,1.0
1,2.0,Badlands,1.0
2,4.0,Denali,1.0
4,6.0,Franklin,1.0
5,7.0,Glacier,1.0
6,8.0,Hampton,1.0
7,9.0,Apollo,4.0
8,11.0,Ike,1.0
9,12.0,Joshua,1.0
10,13.0,Kings,1.0


In [21]:

a=typenamecohort.cohort.to_list()
b=typenamecohort.name.to_list()
c=typenamecohort.program_id.to_list()
chortnamemap=dict(zip(a,b))
chortprogmap=dict(zip(a,c))

metadata.head(2)

,name,cohort_id,start_date,end_date,program_id
0,Andromeda,31.0,2019-03-18,2019-07-30,2.0
1,Apex,56.0,2020-02-24,2020-07-29,2.0


In [22]:



merged=df

merged=merged.fillna(0) 

merged['program_id']=merged['cohort'].map(chortprogmap)
merged['name']=merged['cohort'].map(chortnamemap)


nomenclaturemap={3:'DS',2:'Full_Stack_PHP',1:'Full_Stack_PHP',4:'Front_End'}




merged['program_name']=merged['program_id'].map(nomenclaturemap)


merged['program_name']=merged['program_id'].map(nomenclaturemap)
merged.head()

,page,id,cohort,ip,program_id,name,program_name
datetime,,,,,,,
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,2.0,Teddy,Full_Stack_PHP
2018-01-26 09:56:41,javascript-i/loops,2,22,97.105.19.61,2.0,Teddy,Full_Stack_PHP


In [23]:

merged=merged.merge(metadata,how='left',on=['name','program_id'])
merged

,page,id,cohort,ip,program_id,name,program_name,cohort_id,start_date,end_date
0,java-ii,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP,8.0,2015-09-22,2016-02-06
1,java-ii/object-oriented-programming,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP,8.0,2015-09-22,2016-02-06
2,slides/object_oriented_programming,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP,8.0,2015-09-22,2016-02-06
3,javascript-i/conditionals,2,22,97.105.19.61,2.0,Teddy,Full_Stack_PHP,22.0,2018-01-08,2018-05-17
4,javascript-i/loops,2,22,97.105.19.61,2.0,Teddy,Full_Stack_PHP,22.0,2018-01-08,2018-05-17
...,...,...,...,...,...,...,...,...,...,...
849905,jquery/personal-site,64,28,71.150.217.33,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04
849906,jquery/mapbox-api,64,28,71.150.217.33,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04
849907,jquery/ajax/weather-map,64,28,71.150.217.33,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04
849908,anomaly-detection/discrete-probabilistic-methods,744,28,24.160.137.86,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04


In [24]:



pages=merged.page.value_counts()

y=['.com','.py','.md','.json','.tex','.com','.jpeg','.jpg','.html','.ico','.pn','.png','.svg','.csv']
y=[i.lower().strip() for i in y]


a=list(pages.index)

x=[]
for b in a:
    if type(b)==int:
       
        break
    else:

        blen=len(b)
        b=str(b)
        blist=[]
        b=b.split('/')
        for i in b:
            i=str(i)
            ilist=[]
            for ii in i:
                iilist=[]
                if ii.isnumeric()==False:
                    ii=ii.lower()
                    iilist.append(ii)


                else:
                    iilist.append(ii)
                    continue
                ilist.append(','.join(iilist))
            blist.append(''.join(ilist))
        x.append(''.join(blist))
      
a=x

    



       


    




finalist=[]
for x in a:
    inotx=True
    for i in y:
             
        if i not in x:
            continue
        else:
            inotx=False
    if  inotx==True:
        finalist.append(x)

a=finalist


wordstogroupon=[
'sql','cli',\
'appendix','javascript','java','jquery','stats','spring',\
'timeseries','python','classification','regression','nlp','clustering','sklearn','group','tidy','tree','storytelling',\
'relationships','capstone','storytelling','regex','anom','kmean','advanced','css','html','explore','prep',\
'ajax','git','matplot','prework','quiz','dbscan','php','setup','model','distributed','bokeh','exploration',\
'tableau','seaborn','where','acquisition','spark','joins','distributions','knn','fundamentals','acquire','analysis','bio','merge','ml','effects',\
'wrangle','prophet','forest','subqueries','machinelearning','interview','polymorphism','loops','jupiter','pipeline','types','object',\
'method','instructors','job','grades','json','student','restful','index','bayes','npm','register','mvc','extra','documentation','elements','exceptions','scale',\
'mapbox','cli','eval','correlation','bom','web','team','functions','conditionals','data','passwords','professional','stat','glossary','tools','api','expressions','introduction']
wordstogroupon=list(set(wordstogroupon))

wordslists=[list() for i in wordstogroupon]


wordslist=dict(zip(wordstogroupon,wordslists))
popped=[]
for i in wordslist.keys():
    for x1,x in enumerate(a):
        if i in x:
            u=a.pop(x1)
            popped.append(u)
            wordslist.get(i).append(u)


        
        else:
            continue
a=list(set(a)-set(popped))
wordslist.update({'remainder':a})



wordslist.keys()
tosort=wordslist.get("remainder")



nonalpha=str('1234567890~!%^&*/_-.:;  ')
nonalpha=[(f'{i}') for i in nonalpha]


bsmart=[]
for i in tosort:
    for x in nonalpha:
        i=i.replace(f'{x}','_')
    i=i.replace('_','')
    bsmart.append(i)
bsmart=list(set(bsmart))


smarterlist=[]

for i in bsmart:
    if len(i) <7:
           
        if len(i)==6:
             i=i[:5]

        elif len(i)>5:
            i=i[:4]
        elif len(i)>4:
            i=i[:3]
        elif len(i)>3:
            i=i[:3]
        else:
            continue
        smarterlist.append(i)


smarterlist=list(set(smarterlist))
smarterlist.sort()


from numpy import append


popped=[]
for i in wordslist.keys():
    for x1,x in enumerate(a):
        if i in x:
            u=a.pop(x1)
            popped.append(u)
            wordslist.get(i).append(u)


        
        else:
            continue
a=list(set(a)-set(popped))
wordslist.update({'remainder':a})



keyslist=list(wordslist.keys())

groupsdict=[]
for i in keyslist:
    if len(i)>0:
        x=wordslist.get(i)
        for ii in x:
            groupsdict.append({ii:i})




In [40]:

len(groupsdict)
# Python code to merge dict using a single
# expression
def Merge(dict1, dict2):
	res = {**dict1, **dict2}
	return res
supergroup={}
for i in range(0,len(groupsdict)):
    supergroup=Merge(groupsdict[i],supergroup)





{'sessions-and-cookies': 'remainder',
 '.._entitylabeling': 'remainder',
 '.._normalitytests': 'remainder',
 'f': 'remainder',
 '._create': 'remainder',
 'slidesannotations': 'remainder',
 'slidesdesign_patterns': 'remainder',
 'querypersonal-site': 'remainder',
 '._about_excel': 'remainder',
 '.._flask_intro': 'remainder',
 '_overview': 'remainder',
 'slidesinterfaces_and_abstract_classes': 'remainder',
 'intellij': 'remainder',
 '.-more-exercises': 'remainder',
 '._inspect': 'remainder',
 '.._compare_means': 'remainder',
 'sequel-pro': 'remainder',
 '._intro': 'remainder',
 '_exercises': 'remainder',
 'slidesdeployment_and_dependencies': 'remainder',
 'slidesjpa': 'remainder',
 'arraysmanipulating': 'remainder',
 'codeup-exercises': 'remainder',
 'content..': 'remainder',
 'slidesuser_input': 'remainder',
 '._detect_through_probability': 'remainder',
 'tables': 'remainder',
 '..._prototype': 'remainder',
 '_creating_files_and_directories': 'remainder',
 'slidesjsp_and_jstl': 'remaind

In [26]:
merged.page=merged.page.str.lower()
merged['pagegroups']=merged.page.map(supergroup)

merged['pagegroups'].value_counts()






java               56288
remainder          18925
html               13654
spring             12524
jquery             11525
sql                11041
javascript         10730
appendix            8375
regression          1827
capstone            1291
git                  376
data                 323
web                  316
pipeline             197
classification       191
introduction          63
elements              57
prep                  41
explore               39
acquire               30
css                   26
setup                 23
bayes                 23
clustering            22
stat                  21
functions             20
conditionals          19
loops                 18
mvc                   18
storytelling          17
python                15
object                14
exploration           13
forest                10
php                    9
api                    9
stats                  7
quiz                   7
expressions            6
model                  6


In [27]:
# cohortnames.sort()
# cohortnames


In [28]:

# a=nomenclature['map'].values.tolist()
# b=nomenclature['program_id'].values.tolist()
# mapper=dict(zip(b,a))
# mapper

'Andromeda',
 'Apex',
 'Apollo',

 'Bayes',
 'Betelgeuse',
 'Ceres',
 
 'Denali',
 'Europa',
 'Fortuna',
 'Franklin',
 'Hampton',
 'Ike',
 'Joshua',
 'Lassen',
 'Mammoth',
 'Niagara',
 'Olympic',
 'Pinnacles',
 'Quincy',
 'Sequoia',
 'Staff',
 'Teddy',
 'Ulysses',
 'Wrangell',
 'Xanadu',
 'Yosemite',


In [29]:
#Question 1
# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?



program_traffic=merged.groupby(['program_name'])


question1=pd.DataFrame(program_traffic.page.agg([pd.Series.mode,pd.Series.nunique]));question1



,mode,nunique
program_name,,
DS,search/search_index.json,677
Front_End,content/html-css,3
Full_Stack_PHP,javascript-i,2053


In [30]:
question1=pd.DataFrame(program_traffic.pagegroups.agg([pd.Series.mode,pd.Series.nunique]));question1



,mode,nunique
program_name,,
DS,remainder,38
Front_End,[],0
Full_Stack_PHP,java,65


We can see both specific websites and overallgroups of lessons above.
For the speicif website we have:

DS	search/search_index.json	677
Front_End	content/html-css	3
Full_Stack_PHP	javascript-i	2053

In [43]:
#Question 2
# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?




from statistics import mode


# cohort_traffic=merged.groupby(['name','program_name'])
cohort_traffic=merged.groupby('name')
# kwargs={'ascending':False}
question2a=pd.DataFrame(cohort_traffic.page.agg([pd.Series.mode,pd.Series.nunique]));question2a







,mode,nunique
name,,
Andromeda,toc,365
Apex,search/search_index.json,272
Apollo,content/html-css,3
Arches,javascript-i,341
Badlands,content/php_ii/command-line,43
Bash,search/search_index.json,274
Bayes,1-fundamentals/modern-data-scientist.jpg,529
Betelgeuse,search/search_index.json,333
Ceres,search/search_index.json,300


In [47]:
group2=cohort_traffic.page
top3={}

for n in (group2.groups.keys()):
    x=group2.get_group(n)
    y=x.value_counts().sort_values(ascending=False)
    df=pd.DataFrame(y)
    df.index.rename(name=f'{n}',inplace=True)
    z=(df.head(3).to_dict())
    z=list(z.values())
    z=list(z[0].keys())
    
    top3.update({n:z})
    # print(z)


q2dataframe=pd.DataFrame(top3)

q2dataframe.T


,0,1,2
Andromeda,toc,javascript-i,spring
Apex,search/search_index.json,toc,html-css
Apollo,content/html-css,content/html-css/gitbook/images/favicon.ico,content/html-css/introduction.html
Arches,javascript-i,html-css,javascript-ii
Badlands,content/php_ii/command-line,content/php_i,content/php_ii/control-structures-i
Bash,search/search_index.json,toc,javascript-i
Bayes,1-fundamentals/modern-data-scientist.jpg,1-fundamentals/ai-ml-dl-timeline.jpg,1-fundamentals/1.1-intro-to-data-science
Betelgeuse,search/search_index.json,javascript-i,toc
Ceres,search/search_index.json,javascript-i,toc
Curie,6-regression/1-overview,search/search_index.json,1-fundamentals/modern-data-scientist.jpg


In [33]:

cols=q2dataframe.columns.tolist()
top3set=[]
for i in cols:
    x=q2dataframe[i].tolist()
    top3set.extend(x)

q2dataframe=q2dataframe.T
pd.DataFrame(pd.DataFrame(top3set).value_counts()).head(5)


,0
0,
javascript-i,24
toc,18
search/search_index.json,12
html-css,11
java-iii,10


In [34]:
from numpy import NaN


a='javascript-i'
b='search/search_index.json'
c='toc'
d='html-css'
e='java-iii'

q2dataframe=q2dataframe.applymap(lambda x:np.nan if ((x==a)or (x==b) or(x==c)or (x==d) or(x==e)) else x)
q2dataframe.notna().sum(axis=1).sort_values(ascending=False)[:12]

# toc	18
# search/search_index.json



Joshua      3
Darden      3
Oberon      3
Franklin    3
Florence    3
Quincy      3
Denali      3
Easley      3
Kings       3
Bayes       3
Badlands    3
Apollo      3
dtype: int64

We see that 

Joshua      3
Darden      3
Oberon      3
Franklin    3
Florence    3
Quincy      3
Denali      3
Easley      3
Kings       3
Bayes       3
Badlands    3
Apollo      3

All have websites that are outside the top 4 from codeup overall. As such they spend much more time on these unique lessons in comparison.



Andromeda	toc	365
Apex	search/search_index.json	272
Apollo	content/html-css	3
Arches	javascript-i	341
Badlands	content/php_ii/command-line	43
Bash	search/search_index.json	274
Bayes	1-fundamentals/modern-data-scientist.jpg	529
Betelgeuse	search/search_index.json	333
Ceres	search/search_index.json	300
Curie	6-regression/1-overview	416
Darden	classification/overview	419
Deimos	javascript-i	287
Denali	[mkdocs/search_index.json, prework/databases, prework/versioning/github]	3
Easley	classification/scale_features_or_not.svg	229
Europa	toc	266
Florence	fundamentals/modern-data-scientist.jpg	161
Fortuna	toc	257
Franklin	javascript-ii/es6	44
Ganymede	search/search_index.json	295
Glacier	content/html-css	142
Hampton	java-iii	277
Hyperion	toc	269
Ike	html-css	94
Joshua	content/html-css	116
Jupiter	toc	287
Kalypso	javascript-i	272
Kings	index.html	347
Lassen	index.html	485
Luna	javascript-i	238
Mammoth	java-i	231
Marco	toc	247
Neptune	toc	135
Niagara	spring	134
Oberon	javascript-i/introduction/operators	47
Olympic	javascript-i	382
Pinnacles	javascript-i	324
Quincy	content/laravel/intro	281
Sequoia	spring	311
Staff	javascript-i	1403
Teddy	spring	543
Ulysses	mkdocs/search_index.json	405
Voyageurs	javascript-i	376
Wrangell	toc	420
Xanadu	javascript-i	282
Yosemite	toc	313
Zion	toc	316

In [35]:
#Question 3


In [42]:
#Question 4
suspicious_ip=merged.groupby(['id']).ip


In [ ]:

question4=pd.DataFrame(suspicious_ip.agg([pd.Series.mode,pd.Series.nunique]))
question4=question4.sort_values(by='nunique',ascending=False)
question4=question4[question4['nunique']>=question4['nunique'].quantile(.99)];question4




These are suspicious because they have very high number of ips associated with the same id. They are in the .99 quantile. 




mode	nunique
id		
228	97.105.19.58	148
843	172.58.99.111	122
533	205.251.148.178	91
690	67.11.40.170	86
530	97.105.19.58	86
226	97.105.19.58	83
460	89.187.164.151	81
590	130.45.47.56	74
127	97.105.19.61	61
337	97.105.19.58	56

In [37]:
# 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

merged.groupby

<bound method DataFrame.groupby of                                                     page   id  cohort  \
0                                                java-ii    1       8   
1                    java-ii/object-oriented-programming    1       8   
2                     slides/object_oriented_programming    1       8   
3                              javascript-i/conditionals    2      22   
4                                     javascript-i/loops    2      22   
...                                                  ...  ...     ...   
849905                              jquery/personal-site   64      28   
849906                                 jquery/mapbox-api   64      28   
849907                           jquery/ajax/weather-map   64      28   
849908  anomaly-detection/discrete-probabilistic-methods  744      28   
849909                                 jquery/mapbox-api   64      28   

                   ip  program_id     name    program_name  cohort_id  \
0        97.105

In [38]:
merged


,page,id,cohort,ip,program_id,name,program_name,cohort_id,start_date,end_date,pagegroups
0,java-ii,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP,8.0,2015-09-22,2016-02-06,java
1,java-ii/object-oriented-programming,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP,8.0,2015-09-22,2016-02-06,NaN
2,slides/object_oriented_programming,1,8,97.105.19.61,1.0,Hampton,Full_Stack_PHP,8.0,2015-09-22,2016-02-06,NaN
3,javascript-i/conditionals,2,22,97.105.19.61,2.0,Teddy,Full_Stack_PHP,22.0,2018-01-08,2018-05-17,NaN
4,javascript-i/loops,2,22,97.105.19.61,2.0,Teddy,Full_Stack_PHP,22.0,2018-01-08,2018-05-17,NaN
...,...,...,...,...,...,...,...,...,...,...,...
849905,jquery/personal-site,64,28,71.150.217.33,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04,NaN
849906,jquery/mapbox-api,64,28,71.150.217.33,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04,NaN
849907,jquery/ajax/weather-map,64,28,71.150.217.33,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04,NaN
849908,anomaly-detection/discrete-probabilistic-methods,744,28,24.160.137.86,2.0,Staff,Full_Stack_PHP,28.0,2014-02-04,2014-02-04,NaN


In [39]:
#Question 7

leastaccesed=merged.page.value_counts().sort_values().head(15)
question7=pd.DataFrame(leastaccesed)

question7


	page
appendix/professional-development/post-interview-review-form	1
python_libraries	1
jquery/traversing	1
javascript-/introducion/working-with-data-types-operators-and-variables	1
javascript-/introducion/working-with-dat-types-operators-and-variables	1
coding-challenges	1
capstone/52	1
capstone/53	1
timeline/java-i	1
capstone/54	1
examples/bootstrap/pizza.png;	1
examples/bootstrap/pizza.png%22);	1
professional-development/job-search-workshop	1
examples/bootstrap/pizza.pn	1
users.json	1


IndentationError: unexpected indent (302489382.py, line 9)

In [ ]:
leastaccesedgroups=merged.pagegroups.value_counts().sort_values()[:15]
leastaccesedgroups

We can see that the groups of lessons below had very low hits over all. I removed files that where likely not useful.


types          1
json           1
restful        1
instructors    1
tidy           1
where          1
index          1
joins          1
glossary       1
jupiter        1
matplot        2
api            2
scale          2
job            2
ajax           2




appendix/professional-development/post-interview-review-form	1
python_libraries	1
jquery/traversing	1
javascript-/introducion/working-with-data-types-operators-and-variables	1
javascript-/introducion/working-with-dat-types-operators-and-variables	1
coding-challenges	1
capstone/52	1
capstone/53	1
timeline/java-i	1
capstone/54	1
examples/bootstrap/pizza.png;	1
examples/bootstrap/pizza.png%22);	1
professional-development/job-search-workshop	1
examples/bootstrap/pizza.pn	1
users.json	1

In [ ]:









question7=question7.page.nsmallest(n=1,keep='all')




question7=question7.index


question7.sort()

In [ ]:

# for i in cols:
           
#           fig,axs=plt.subplots(1,3) 

#           print(i,axs,'\n')
#           kwargs={'family':'sans-serif'} 
#           fig.suptitle(t=f'{i}:\n [Hist w/ Kde] v. [Box] v. [Q-Q] ',**kwargs)
        
#           sns.histplot(x=i,data=df, kde=True,ax=axs[0])
#           print(i) 
#           sns.boxplot(x=i,data=df,ax=axs[1])
#           axs[2]=stats.probplot(df[i],dist='norm',plot=plt)


#           fig.set_figheight(8)
#           fig.set_figwidth(32)

#           plt.show()
#           print("\n\n\n")


# infodf=pd_DF_one_shot_info(df)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.page)
